In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/6x/2v0z1h_n6vj6jkt8s3sjsbv40000gn/T/ipykernel_76777/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv('/Users/iz/Desktop/incomplete labs/lab-natural-language-processing/data/kg_train.csv',encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
# Your code

#from sklearn.model_selection import train_test_split

# Separate features (X) and labels (y)
#X = data['text']  
#y = data['label']

# Split the data (20% for testing)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print shapes
#print(f'Train size: {X_train.shape}, Test size: {X_test.shape}')


In [5]:
#print(test_df.columns)


In [6]:
#Note: The following code assumes that the dataset has already been split into train and test sets as specified by instructor. So now I'm not resplitting hte data.

# Load the already-split datasets
train_df = pd.read_csv('/Users/iz/Desktop/incomplete labs/lab-natural-language-processing/data/kg_train.csv', encoding='latin-1')
test_df = pd.read_csv('/Users/iz/Desktop/incomplete labs/lab-natural-language-processing/data/kg_test.csv', encoding='latin-1')

# Optional: reduce size to speed up development (for training only)
train_df = train_df.head(1000)

# Handle missing values
train_df.fillna("", inplace=True)
test_df.fillna("", inplace=True)

# Assign preprocessed variables
X_train = train_df['text']
y_train = train_df['label']

X_test = test_df['text']
#y_test = test_df['label']

# Confirm shape
print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")


Train size: (1000,), Test size: (5964,)


## Data Preprocessing

In [7]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
# Your code
!pip install beautifulsoup4

from bs4 import BeautifulSoup

def clean_html_bs(text):
    # Parse the HTML
    soup = BeautifulSoup(text, "html.parser")

    # Remove script and style elements
    for script_or_style in soup(["script", "style"]):
        script_or_style.extract()

    # strips the rest of the HTML tags by converting parsed object back to plain text with .get_text()
        # Get text content only (strips all tags)
    cleaned_text = soup.get_text()

    return cleaned_text

# Apply to the training and test sets
X_train = X_train.apply(clean_html_bs)
X_test = X_test.apply(clean_html_bs)




/var/folders/6x/2v0z1h_n6vj6jkt8s3sjsbv40000gn/T/ipykernel_76777/2677811227.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/var/folders/6x/2v0z1h_n6vj6jkt8s3sjsbv40000gn/T/ipykernel_76777/2677811227.py:8: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Pretokenization preprocessing (cleaning data before spliting words into tokens, lammatize, and split). This is specific to html. You can remove the comments after you remove the javascript

pre-tokenization preprocessing:
- Remove all the special characters - dont need special NLP methods. Can use string methods (lambda, etc of the corresponding columns you want to change. String methods are from the beginning in the functions labs. Like to identify special characters)
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [9]:
# Your code
import re

def clean_text(text):
    text = re.sub(r'\W', ' ', text)               # Remove all non-word characters (keep only letters, numbers, _)
    text = re.sub(r'\d+', '', text)               # Remove numbers
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)   # Remove single characters between spaces (e.g., ' a ')
    text = re.sub(r'^[a-zA-Z]\s+', '', text)      # Remove single characters at start
    text = re.sub(r'\s+', ' ', text)              # Replace multiple spaces with a single space
    text = re.sub(r"^b[\'\"]", '', text)          # Remove prefix b' or b" at start 
    return text.lower()                           # Convert to lowercase

# Apply to cleaned HTML text
X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)


## Now let's work on removing stopwords
Remove the stopwords.

- from here NLP methods we learned

In [10]:
import nltk
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


False

In [11]:
from nltk.corpus import stopwords

# Load the English stopword list
stop_words = set(stopwords.words('english'))

#loop version to remove stop words
def remove_stopwords(text): #defines the function to remove stopwords (takes a string as input and returns a string without stopwords)
    words = text.split()  # Split sentence into words
    filtered = []          # Create empty list to hold non-stop words
    
    for word in words: #loop through each word, 
        if word not in stop_words: # If the word is not in the stopword list, append it to filtered,
            filtered.append(word) #append it to the filtered list
    
    return ' '.join(filtered)  # Join the filtered words back into a sentence with space separations

#list comprehension method to remove stopwords
#def remove_stopwords(text):
    #words = text.split() #tokenizes the text into words (splits into words)
    #filtered = [word for word in words if word not in stop_words] #Filters out words that are in the stopword list
    #return ' '.join(filtered) #Rejoins the filtered words into a cleaned string

# Apply remove stopwords function to both training and test sets
X_train = X_train.apply(remove_stopwords)
X_test = X_test.apply(remove_stopwords)


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [12]:
# Your code

import nltk
from nltk.stem import WordNetLemmatizer

# Download wordnet if not already done
nltk.download('wordnet')
nltk.download('omw-1.4')  # For extended lemmatization

lemmatizer = WordNetLemmatizer() # Initialize the lemmatizer

def lemmatize_text(text): #takes a string as input and returns a lemmatized string
    words = text.split() #split the sentance/text into words
    lemmatized = [] # Initialize an empty list to store lemmatized words
    for word in words: #loop through each word in the text/sentane
        lemmatized_word = lemmatizer.lemmatize(word) # Lemmatize the word. Works by converting each word to its lemma base form using wordnet dictionary. Default (wihout POS) is noun
        lemmatized.append(lemmatized_word) # Append the lemmatized word to the list
    return ' '.join(lemmatized) #join the lemmatized words back into a single string with space separations

# Apply to your cleaned and stopword-filtered text
X_train = X_train.apply(lemmatize_text)
X_test = X_test.apply(lemmatize_text)


[nltk_data] Error loading wordnet: <urlopen error [Errno 54]
[nltk_data]     Connection reset by peer>
[nltk_data] Downloading package omw-1.4 to /Users/iz/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [13]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Combine text and labels into a DataFrame for easier manipulation
train_df = pd.DataFrame({'text': X_train, 'label': y_train})

# Remove rows with empty or whitespace-only text
train_df = train_df[train_df['text'].str.strip() != '']

# Separate the df into 2 parts: ham and spam texts
ham_texts = train_df[train_df['label'] == 'ham']['text']
spam_texts = train_df[train_df['label'] == 'spam']['text']

# EDA portion: Check if either set is empty to prevent errors in vectorizaton, otherwise vectorize (turn words into frequency counts using countvectorizer), then convert into a dataframe and sum word counts across all messages in that label group, then sort and shot the top 10 most common words.
#fit_transform... = sparse matrix of word counts
#.toarray() = numpy matrix (an array)
#pd.datafram... = converted numpy matrix/array into a word counts dataframe (labeled, pandas dataframe/table)
#.sort..= top 10 mst common words.
if ham_texts.empty:
    print("No valid ham messages after preprocessing.")
else:
    vectorizer_ham = CountVectorizer()
    ham_matrix = vectorizer_ham.fit_transform(ham_texts)
    ham_counts = pd.DataFrame(ham_matrix.toarray(), columns=vectorizer_ham.get_feature_names_out()).sum().sort_values(ascending=False)
    print("Top 10 Ham Words:")
    print(ham_counts.head(10))

if spam_texts.empty:
    print("No valid spam messages after preprocessing.")
else:
    vectorizer_spam = CountVectorizer()
    spam_matrix = vectorizer_spam.fit_transform(spam_texts)
    spam_counts = pd.DataFrame(spam_matrix.toarray(), columns=vectorizer_spam.get_feature_names_out()).sum().sort_values(ascending=False)
    print("\nTop 10 Spam Words:")
    print(spam_counts.head(10))


No valid ham messages after preprocessing.
No valid spam messages after preprocessing.


## Extra features

In [14]:
# in order to run the below code cell (data_trrain/val was not defined), I need to Create data_train and data_val from cleaned data in order to run the next code cell.
# Create data_train and data_val using cleaned text
data_train = pd.DataFrame({'preprocessed_text': X_train, 'label': y_train})
data_val = pd.DataFrame({'preprocessed_text': X_test})  # no label in test set


print("Training Data Sample:")
print(data_train.head())
print("\nValidation Data Sample:")
print(data_val.head())



Training Data Sample:
                                   preprocessed_text  label
0  dear sir strictly private business proposal mi...      1
1                                                         0
2  nora cheryl emailed dozen memo haiti weekend p...      0
3  dear sir fmadam know proposal might surprise e...      1
4                                                fyi      0

Validation Data Sample:
                                   preprocessed_text
0  usiness fact deceased man foreigner authorized...
1  happy adjust afternoon going suggest pm start ...
2  lael brainard confirmed afternoon miguel rodri...
3  friday march sbwhoeopã â ã â ã â extended cong...
4  dear good friend happy inform succe ssin getti...


In [15]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,preprocessed_text,label,money_mark,suspicious_words,text_len
0,dear sir strictly private business proposal mi...,1,1,1,1493
1,,0,1,0,0
2,nora cheryl emailed dozen memo haiti weekend p...,0,1,0,111
3,dear sir fmadam know proposal might surprise e...,1,1,1,1350
4,fyi,0,1,0,3


## How would work the Bag of Words with Count Vectorizer concept?
- Bag of words works by converting words/text/docs into vectors (numeric data).
- Bag of words works by: tokenizing the text, counting word fre4quencies,and then representing each document as a row of word counts. 
- essentially, each document becomes a vector of word freequenies (which are now numeric data), forming a matrix of numbers (one row per document, one column per word) that the model can use to count.

1. Start with a list of texts (the documents).
2. Build a vocabulary of all unique words across all the docs.
3. For each document, count how many times each word in the vocabulary appears.

Below is an example of how this works

In [16]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

#1. start with a list of texts (documents)
# Sample documents
corpus = ["free money now", "call me now", "free offer"]

#2. Build the vocabulary of unique words from all the documents
# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the text
X = vectorizer.fit_transform(corpus)

# Get feature names
print("Vocabulary:", vectorizer.get_feature_names_out())

#3. count number of times each word apperas in each document and create a matrix where each row is a document and each column is a word from the vocabulary.
# View the Bag of Words matrix
print("\nBag of Words Matrix:\n", X.toarray())


Vocabulary: ['call' 'free' 'me' 'money' 'now' 'offer']

Bag of Words Matrix:
 [[0 1 0 1 1 0]
 [1 0 1 0 1 0]
 [0 1 0 0 0 1]]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [17]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize and fit TF-IDF vectorizer on training data
tfidf_vectorizer = TfidfVectorizer()

# Fit on train, transform both
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Shape info
print("TF-IDF shape (train):", X_train_tfidf.shape)
print("TF-IDF shape (test):", X_test_tfidf.shape)


TF-IDF shape (train): (1000, 22015)
TF-IDF shape (test): (5964, 22015)


## And the Train a Classifier?
- tp distinguish between 0s (ham) and 1s (spam)
- use which ever classifier you want. You dont have to hyperparamter tune it its just proof of concept-training classifier for binary classification.

In [20]:
# Your code
from sklearn.linear_model import LogisticRegression

# Train model
model_tfidf = LogisticRegression()
model_tfidf.fit(X_train_tfidf, y_train)

# Predict on test set
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# Show predictions (you don’t have y_test)
print("First 10 predictions on test set:")
print(y_pred_tfidf[:10])


First 10 predictions on test set:
[1 0 0 0 1 1 0 1 1 1]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [21]:
# Your code
from sklearn.naive_bayes import MultinomialNB

# Initialize the classifier (default parameters)
nb_classifier = MultinomialNB()

# Train on TF-IDF-transformed training data
nb_classifier.fit(X_train_tfidf, y_train)

# Predict on test set (you don’t have y_test — just show preds)
nb_predictions = nb_classifier.predict(X_test_tfidf)

# Preview predictions
print("MultinomialNB predictions on test set (first 10):")
print(nb_predictions[:10])


MultinomialNB predictions on test set (first 10):
[1 0 0 0 1 1 0 1 1 1]


Othe rimportant:
- vectorizer best practice is like with scalers: you fit it to your traning data and transform entire data. Dont fit to training and test, only fit to training and then apply it to train and test (to avoid data leakage). basically only fit to training then apply to train and test.
- pretokenization cleaning you apply to both sets, training and test. 

- one of the labels....basically looking for predominant words appearing only in spam and not so predominant in ham. 
- you can do plots to visualize dist of data and frequency of words. 

- keep the steps from the lab as a quick reference guide for cleaning HTML data...might not need all the steps but good to be aware. 